<a href="https://colab.research.google.com/github/Kumarvels/GenAIProjects/blob/main/LangGraph_to_build_AI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Building an AI Agent with LangGraph in Google Colab
# Usecase for to build a simple AI agent using LangGraph to calculate solar  panel energy savings
This notebook demonstrates how to build a simple AI agent using LangGraph to calculate solar panel energy savings. The agent will:
- Interact with the user to gather inputs (e.g., electricity cost).
- Use a tool node to compute savings.
- Display the results.



2: Install Required Libraries

Step 2.1: Install LangChain, LangGraph, and Other Dependencies
In a new code cell, run the following command to install the required libraries

Explanation:
langchain: Core library for building language model workflows.

langgraph: Extension for creating stateful, multi-agent workflows with cycles.

langchain-community: Community-contributed tools and integrations.

langchain-core: Core components of LangChain.

langchain-openai: Integration with OpenAI models (we’ll use this for the language model).

python-dotenv: For managing environment variables (e.g., API keys).




In [ ]:
!pip install langchain langgraph langchain-community langchain-core langchain-openai python-dotenv

Step 2.2: Install Additional Tools (Optional)

If you plan to use Google’s Gemini model, you’ll need the Google Generative AI package





In [ ]:
!pip install langchain-google-genai

Step 2.3: Verify Installation

Add a code cell to check the installed versions

LangChain and LangGraph are frameworks for building applications that use large language models (LLMs), but they differ in their approach to workflow management.

LangChain is designed for simple, linear workflows

LangGraph focuses on complex, graph-based workflows, making it suitable for agentic and multi-actor scenarios

Key Differences:

Workflow Architecture:
LangChain uses a linear, chain-based architecture, while LangGraph uses a graph-based architecture, allowing for more complex and dynamic workflows.

Agentic Systems:
LangGraph is particularly well-suited for building agentic systems, where multiple agents interact and collaborate, while LangChain is more focused on simpler, sequential tasks.

State Management:
LangGraph provides built-in support for managing state, which is crucial for complex workflows where context and information need to be carried across different steps.

Flexibility:
LangGraph offers greater flexibility and control over workflow design, allowing for dynamic branching, loops, and other complex control flow patterns.

Scalability:
Both frameworks are designed to be scalable, but LangGraph's graph-based architecture allows for more easily handling complex workflows with multiple agents and interactions.

In [ ]:
import langchain
import langgraph
print("LangChain version:", langchain.__version__)

3: Set Up API Keys and Environment Variables

Step 3.1: Obtain an OpenAI API Key
Go to OpenAI, sign up/log in, and generate an API key.

Alternatively, if using Google’s Gemini model, get an API key from Google AI Studio.

Step 3.2: Store the API Key Securely in Colab
Use Google Colab’s user input feature to securely input your API key without hardcoding it



In [ ]:
from getpass import getpass
import os

# Prompt for OpenAI API key
# openai_api_key = getpass("Enter your OpenAI API key: ")
# os.environ["OPENAI_API_KEY"] = openai_api_key

# If using Google Gemini, uncomment the following:
google_api_key = getpass("Enter your Google API key: ")
os.environ["GOOGLE_API_KEY"] = google_api_key

4: Define the State and Tools for the AI Agent

Step 4.1: Import Required Modules
Add a code cell to import the necessary LangChain and LangGraph components



In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.tools import tool

4.2: Define the State

LangGraph uses a state to keep track of the agent’s progress. Define a state class to store messages and user inputs

Explanation:

messages: A list of conversation messages (human and AI).

electricity_cost: The user’s electricity cost per kWh (to be provided by the user).

savings: The calculated savings from using solar panels.

Annotated with operator.add allows messages to be appended to the list.




In [ ]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    electricity_cost: float  # Store the user's electricity cost
    savings: float  # Store the calculated savings

4.3: Create a Tool for Calculating Solar Savings

Define a tool that calculates potential savings based on the user’s electricity cost

Explanation:

The @tool decorator makes this function a LangChain tool.

We assume a 5 kW solar system produces 20 kWh/day (a rough estimate).

Savings are calculated as: yearly production (kWh) × cost per kWh.



In [ ]:
from typing import Dict, Any
import inspect
from pydantic import BaseModel, Field

# Define a Pydantic model for the tool's input
class SolarSavingsInput(BaseModel):
    electricity_cost: float = Field(description="The cost of electricity per kWh in dollars.")

# Remove the @tool decorator so the LLM doesn't see this as a callable tool
# @tool
def calculate_solar_savings_manual(args: SolarSavingsInput) -> float:
    """
    Calculate potential solar panel savings based on electricity cost per kWh.
    Accepts arguments as a SolarSavingsInput model.
    Assumes a 5 kW solar system producing 20 kWh/day, 365 days/year.
    This function is intended for manual execution in the tool_node.
    """
    # Optional: Print function signature for debugging
    print(f"Debug: calculate_solar_savings_manual signature: {inspect.signature(calculate_solar_savings_manual)}")


    electricity_cost = args.electricity_cost # Access cost from the Pydantic model
    try:
        cost = float(electricity_cost)
        if cost <= 0:
             raise ValueError("Electricity cost must be positive.")
    except (ValueError, TypeError):
        raise ValueError(f"Invalid electricity cost provided: {electricity_cost}. Must be a number.")

    daily_production = 20  # kWh per day
    yearly_production = daily_production * 365  # kWh per year
    savings = yearly_production * cost  # Yearly savings in dollars
    return round(savings, 2)

5: Build the LangGraph Workflow

Step 5.1: Initialize the Language Model
Set up the language model (e.g., OpenAI’s GPT,  Google Gemini Pro/Flash models)



In [ ]:
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm_with_tools = llm.bind_tools([calculate_solar_savings])
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0, google_api_key=os.environ["GOOGLE_API_KEY"])
# Remove the tool binding as we are manually handling the tool execution
# llm_with_tools = llm.bind_tools([calculate_solar_savings])
llm_with_tools = llm # Use the plain LLM without tools bound
print("Google Gemini model initialized.")

6: Define the Assistant and Tool Nodes

What: Create the nodes for the LangGraph workflow (assistant and tool nodes).

Why: Nodes represent the steps in the agent's workflow: interaction (assistant) and computation (tool).

How: Define functions for each node to handle user input and tool execution.

Outcome: The agent can interact with the user and calculate savings in a structured workflow.


In [ ]:
from langchain_core.messages import ToolMessage # Import ToolMessage
# Removed: import inspect (not needed in this cell)
# Removed: from pydantic import BaseModel, Field (SolarSavingsInput is defined in 9z0SW9IXHI2b)

# The SolarSavingsInput Pydantic model is defined in cell 9z0SW9IXHI2b.
# The @tool calculate_solar_savings function was previously defined and bound, but
# we are now removing the @tool decorator in cell 9z0SW7IXHI2b and handling
# the calculation manually in the tool_node.

# Removed the redundant definition of calculate_solar_savings and SolarSavingsInput
# from this cell. They are defined in cell 9z0SW7IXHI2b.


def assistant_node(state: AgentState) -> AgentState:
    """
    Assistant node: Handles user interaction, attempts to parse electricity cost,
    and invokes the language model if necessary.
    """
    messages = state["messages"]
    electricity_cost = state.get("electricity_cost", None)

    # If electricity_cost is not set, try to parse it from the last human message
    if electricity_cost is None:
        last_human_message = None
        # Find the latest human message
        for msg in reversed(messages):
            if isinstance(msg, HumanMessage):
                last_human_message = msg
                break

        if last_human_message:
            try:
                # Attempt to parse the content as a float
                cost = float(last_human_message.content)
                if cost > 0:
                    # If successful and positive, update the state with the parsed cost
                    print(f"Parsed electricity cost: {cost}. Updating state.") # Debug print
                    # Return the state with the updated electricity_cost, without involving the LLM
                    return {
                        "messages": messages, # Do not add a tool call message here
                        "electricity_cost": cost, # Update the state
                        "savings": state.get("savings", 0.0)
                    }
                else:
                     # If not positive, add a message asking for a positive value
                    print("Parsed non-positive cost, asking again.") # Debug print
                    # In this case, we still involve the LLM to generate the response asking for a positive value
                    response = llm_with_tools.invoke(messages + [AIMessage(content="Please provide a positive electricity cost per kWh (in dollars).")])
                    return {
                        "messages": messages + [response],
                        "electricity_cost": None, # Keep it None as valid cost wasn't provided
                        "savings": state.get("savings", 0.0)
                    }
            except (ValueError, TypeError):
                # If parsing fails, proceed to involve the LLM
                print("Parsing failed, involving LLM.") # Debug print
                pass # Continue below to involve LLM


    # If electricity_cost is already set (meaning we've already parsed it and likely generated a tool call),
    # or if parsing failed and we need the LLM to respond to the initial query or an unparseable input.
    # Involve the LLM to generate the next message.
    print("Invoking LLM.") # Debug print
    response = llm_with_tools.invoke(messages)
    return {"messages": messages + [response]}


def tool_node(state: AgentState) -> AgentState:
    """
    Tool node: Executes the solar savings calculation based on the electricity cost
    provided in the state. Manually performs the calculation using the function
    defined in cell 9z0SW7IXHI2b.
    """
    print("Entering tool_node.") # Debug print
    messages = state["messages"]
    electricity_cost = state.get("electricity_cost")

    if electricity_cost is not None:
        try:
            # Manually extract and validate the electricity_cost from the state
            cost = float(electricity_cost)
            if cost <= 0:
                 raise ValueError("Electricity cost must be positive.")

            # Use the manual calculation function defined in cell 9z0SW7IXHI2b
            # We need to pass the cost as a SolarSavingsInput object
            savings = calculate_solar_savings_manual(args=SolarSavingsInput(electricity_cost=cost))

            # Create a message with the result
            result_message = AIMessage(content=f"Based on your electricity cost of ${cost} per kWh, the estimated yearly solar panel savings are ${savings}.")
            print(f"Calculated savings: ${savings}") # Debug print


            # Update the state with the calculated savings and the cost used
            return {
                "messages": messages + [result_message],
                "electricity_cost": cost, # Keep the cost in state
                "savings": savings
            }
        except Exception as e:
            # Handle errors during manual calculation or argument extraction/validation
            error_message = AIMessage(content=f"Error processing electricity cost or performing calculation: {e}")
            print(f"Error in tool_node during manual calculation: {e}") # Debug print
            return {"messages": messages + [error_message]}
    else:
        # This case should ideally not happen if routed correctly by should_continue
        print("Electricity cost not found in state in tool_node.") # Debug print
        return {"messages": messages + [AIMessage(content="Could not find electricity cost in the state to perform calculation.")]}


print("Nodes defined.")

7: Build and Compile the LangGraph Workflow
  
What: Construct the LangGraph workflow by connecting nodes with edges.

Why: The workflow defines the agent's behavior: start with the assistant, move to the tool, and end.

How: Use StateGraph to add nodes, set the entry point, and define conditional edges.

Outcome: A compiled graph ready to process user inputs and calculate savings.


In [ ]:
builder = StateGraph(AgentState)

# Add nodes
builder.add_node("assistant", assistant_node)
builder.add_node("tools", tool_node)

# Define the entry point
builder.set_entry_point("assistant")

# Define a function to decide whether to continue or end
def should_continue(state: AgentState) -> str:
    # If electricity_cost is set in the state, move to tools
    if state.get("electricity_cost") is not None:
        print("Routing to tools node based on electricity_cost in state.") # Debug print
        return "tools"
    messages = state["messages"]
    last_message = messages[-1]
    # If the assistant returned a tool call (which shouldn't happen with the fix),
    # or if the last message is from the assistant and does not have tool calls,
    # it means the assistant has responded with a non-tool call message, so we end.
    # If the last message is a HumanMessage, the assistant node will be called again.
    if isinstance(last_message, AIMessage) and not last_message.tool_calls:
         print("Routing to end as assistant responded without tool calls.") # Debug print
         return "end"
    # Otherwise, continue to the assistant node (e.g., if the last message was HumanMessage)
    else:
        print("Routing back to assistant.") # Debug print
        return "assistant"


# Add conditional edge from assistant
builder.add_conditional_edges(
    "assistant",
    should_continue,
    {"tools": "tools", "end": END, "assistant": "assistant"} # Add 'assistant' as a possible route
)

# Add edge from tools to end
builder.add_edge("tools", END)

# Compile the graph
graph = builder.compile()
print("Graph compiled successfully.")

8: Run the AI Agent

What: Simulate a user interaction to test the AI agent.

Why: Running the agent demonstrates its functionality and ensures it works as expected.

How: Provide an initial message, run the graph, and simulate providing the electricity cost.

Outcome: The agent prompts for the cost, calculates savings, and displays the result (e.g., $1095 for $0.15/kWh).


In [ ]:
# Step 1: Start the conversation - User asks to calculate savings
initial_state = {
    "messages": [HumanMessage(content="I want to calculate my solar panel savings.")],
    "electricity_cost": None,
    "savings": 0.0
}

print("\n--- First Interaction ---")
# Use list(graph.stream(initial_state)) to collect all events
events1 = list(graph.stream(initial_state))
for event in events1:
    for key, value in event.items():
        print(f"Node: {key}")
        if "messages" in value:
            # Print the content of the latest message
            print(value["messages"][-1].content)
        if "electricity_cost" in value:
            print(f"Electricity Cost: {value['electricity_cost']}")
        if "savings" in value:
             print(f"Savings: {value['savings']}")


# Step 2: Continue the conversation - User provides the electricity cost
# To simulate the next turn, we need the final state from the previous interaction.
# The last event in events1 contains the final state after the first interaction.
last_event_step1 = events1[-1]
# Get the state from the last event by accessing the dictionary value
# The key might be a node name or END
last_state_step1 = list(last_event_step1.values())[0]


print("\n--- Second Interaction (with cost provided) ---")
# Now, add the user's response with the cost to the messages from the previous state
state_with_cost = {
    "messages": last_state_step1["messages"] + [HumanMessage(content="0.15")],
    "electricity_cost": last_state_step1["electricity_cost"], # This should still be None from step 1
    "savings": last_state_step1["savings"]
}

# Use list(graph.stream(state_with_cost)) to collect all events for the second interaction
events2 = list(graph.stream(state_with_cost))
for event in events2:
    for key, value in event.items():
        print(f"Node: {key}")
        if "messages" in value:
            # Print the content of the latest message
            print(value["messages"][-1].content)
        if "electricity_cost" in value:
            print(f"Electricity Cost: {value['electricity_cost']}")
        if "savings" in value:
             print(f"Savings: {value['savings']}")